### Deep Learning Model

Generate a deep learning model that can help to identify whether or not a person is likely to depart from the company given his or her current employee profile.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


### Preprocessing

There are multiple columns with categorical values. Generate a list of categorical variable names.

In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

There are 8 categorical variables that need encoding. Before looping through these variables and encode them using Scikit-learn’s OneHotEncoder, make sure that none of the categorical variables have more than 10 unique values and require bucketing.

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

According to the nunique method, none of the categorical variables have more than 10 unique values, which means we’re ready to encode.

Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time. The only difference from our single variable example is we need to pass our attrition_cat variable list.

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset.

In [5]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df, left_index = True, right_index = True)
attrition_df = attrition_df.drop(attrition_cat, 1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
attrition_df.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
       'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Attrition_No',
       'Attrition_Yes', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Human Resources', 'Department_Research & Development',
       'Department_Sales', 'EducationField_Human Resources',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'EducationField_Medical', 'EducationField_Other',
       'EducationField_Technical Degree', 'Gen

Split into training and testing data, then standardize the numerical variables using Scikit-learn’s StandardScaler module.

NOTE: We need to split our training and testing data before fitting our StandardScaler instance. This prevents testing data from influencing the standardization function.

In [7]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78)

NOTE: For our purposes, we want to build a model that will predict whether or not a person is at risk for attrition; therefore, we must separate the “Attrition” columns from the rest of the input data. In fact, because the attrition data is dichotomous (one of two values), we only need to keep the “Attrition_Yes” column. We can ignore the “Attrition_No” column because it is redundant. 

Now that training and testing data have been allocated, we’re ready to build our StandardScaler object and standardize the numerical features.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Build Deep Learning Module

Data is preprocessed and separated and ready for modelling.

Framework for our basic neural network:
* For input layer, add the number of input features equal to the number of variables in our feature DataFrame.
* For hidden layers, our deep learning model structure will be slightly different.
    * Add two hidden layers with only a few neurons in each layer. 
    * To create the second hidden layer, add another Keras Dense class while defining our model. 
    * All hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
* For output layer, use the same parameters from the basic neural network including the sigmoid activation function. The sigmoid activation function will helps predict the probability that an employee is at risk for attrition.

In [9]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, 
                             input_dim = number_input_features, 
                             activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, 
                             activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, 
                             activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


Time to compile our model and define the loss and accuracy metrics. Since we want to use our model as a binary classifier, we’ll use the binary_crossentropy loss function, adam optimizer, and accuracy metrics, which are the same parameters we used for our basic neural network.

In [10]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

### Train and evaluate the deep learning model

In [11]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs = 100)

Train on 1102 samples
Epoch 1/100
1102/1102 [==============================] - 1s 1ms/sample - loss: 1155.7449 - accuracy: 0.1724
Epoch 2/100
1102/1102 [==============================] - 0s 87us/sample - loss: 666.5473 - accuracy: 0.1797
Epoch 3/100
1102/1102 [==============================] - 0s 84us/sample - loss: 394.6199 - accuracy: 0.2033
Epoch 4/100
1102/1102 [==============================] - 0s 89us/sample - loss: 230.2399 - accuracy: 0.3103
Epoch 5/100
1102/1102 [==============================] - 0s 87us/sample - loss: 118.2854 - accuracy: 0.3838
Epoch 6/100
1102/1102 [==============================] - 0s 199us/sample - loss: 26.7278 - accuracy: 0.4310
Epoch 7/100
1102/1102 [==============================] - 0s 92us/sample - loss: 5.6245 - accuracy: 0.6661
Epoch 8/100
1102/1102 [==============================] - 0s 85us/sample - loss: 4.6001 - accuracy: 0.6461
Epoch 9/100
1102/1102 [==============================] - 0s 89us/sample - loss: 4.0343 - accuracy: 0.6379
Epoch 10/100

1102/1102 [==============================] - 0s 132us/sample - loss: 0.5097 - accuracy: 0.8058
Epoch 77/100
1102/1102 [==============================] - 0s 126us/sample - loss: 0.4930 - accuracy: 0.8094
Epoch 78/100
1102/1102 [==============================] - 0s 134us/sample - loss: 0.4952 - accuracy: 0.8113
Epoch 79/100
1102/1102 [==============================] - 0s 139us/sample - loss: 0.4862 - accuracy: 0.8113
Epoch 80/100
1102/1102 [==============================] - 0s 132us/sample - loss: 0.4691 - accuracy: 0.8131
Epoch 81/100
1102/1102 [==============================] - 0s 123us/sample - loss: 0.4670 - accuracy: 0.8185
Epoch 82/100
1102/1102 [==============================] - 0s 116us/sample - loss: 0.4846 - accuracy: 0.8203
Epoch 83/100
1102/1102 [==============================] - 0s 124us/sample - loss: 0.4935 - accuracy: 0.8113
Epoch 84/100
1102/1102 [==============================] - 0s 127us/sample - loss: 0.5208 - accuracy: 0.8013
Epoch 85/100
1102/1102 [=================

Now that our deep learning model is properly trained, we can evaluate the model’s performance by testing its predictive capabilities on our testing dataset.

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.7900 - accuracy: 0.8614
Loss: 0.8351342004278431, Accuracy: 0.8614130616188049


TAKEAWAY: Looking at our deep learning model’s performance metrics, the model was able to correctly identify employees who are at risk of attrition approximately 87% of the time. Considering that our input data included more than 30 different variables with more than 1,400 data points, the deep learning model was able to produce a fairly reliable classifier.